In [16]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from googletrans import Translator
translator = Translator()

# 1. 데이터 로드
with open('own_dataset_video_test/output/frame_output_v.json', 'r') as file:
    frame_data = json.load(file)

# 2. 텍스트 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. 프레임 캡션 임베딩 생성
captions = [frame["caption"] for frame in frame_data]
caption_embeddings = model.encode(captions, convert_to_tensor=False)

# 4. FAISS 인덱스 생성
embedding_dim = caption_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(caption_embeddings))

# 5. 질문 처리 및 검색
def search(query, top_k=1):
    # 질문 임베딩 생성
    query_embedding = model.encode([query], convert_to_tensor=False)
    # 유사도 검색
    distances, indices = index.search(np.array(query_embedding), top_k)
    
    # 검색 결과 반환
    results = []
    for idx in indices[0]:
        results.append({
            "video_id": frame_data[idx]["video_id"],
            "timestamp": frame_data[idx]["timestamp"],
            "caption_ko": frame_data[idx]["caption_ko"]
        })
    return results

# 6. 테스트 실행
query = "바위 위에 두 사람이 앉아 있는 장면."
query_en = translator.translate(query, dest='en').text
results = search(query_en, top_k=1)

# 결과 출력
for result in results:
    print(f"Query: {query}")
    print(f"Video ID: {result['video_id']}, Timestamp: {result['timestamp']}")
    print(f"Caption_ko: {result['caption_ko']}")

Query: 두 명의 스쿠버다이버가 있는 장면.
Video ID: -ncFDuKdgNE, Timestamp: 0:01:12.0
Caption_ko: 이미지에는 두 명의 스쿠버 다이버가 있는 수중 장면이 묘사되어 있습니다. 물은 탁하고 거품이 표면으로 올라와 움직임을 나타냅니다. 왼쪽의 다이버는 빨간색 다이빙 마스크를 착용하고 있으며 카메라나 조명과 같은 장비를 들고 있는 것처럼 보입니다. 오른쪽 다이버는 노란색 다이빙 마스크를 착용하고 장비도 들고 있습니다. 물의 투명도와 거품의 존재로 인해 전체적인 분위기가 다소 흐릿합니다. 이미지의 왼쪽 하단에 "MOVIECLIPS.COM"이라는 워터마크가 있습니다.
